<a href="https://colab.research.google.com/github/Sadman26/FUZZY/blob/master/InceptionV3_Strawberry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
from google.colab import drive
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3

In [47]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [49]:
train_dir = '/content/drive/MyDrive/strawberry/sl_train'
validation_dir = '/content/drive/MyDrive/strawberry/sl_test'
predict_dir = '/content/drive/MyDrive/strawberry/sl_valid'

In [50]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [51]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)


In [52]:
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=IMG_SIZE,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              target_size=IMG_SIZE,
                                                              batch_size=BATCH_SIZE,
                                                              class_mode='categorical')

Found 775 images belonging to 3 classes.
Found 689 images belonging to 3 classes.


In [53]:
# Load the InceptionV3 model and freeze its layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
for layer in base_model.layers:
    layer.trainable = False

In [57]:
# Add a custom classifier on top of the base model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [58]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [60]:
# Train the model
history = model.fit(train_generator,
                    epochs=10, 
                    validation_data=validation_generator)

Epoch 1/10
25/25 [==============================] - 213s 9s/step - loss: 0.0297 - accuracy: 0.9923 - val_loss: 0.0274 - val_accuracy: 0.9971
Epoch 2/10
25/25 [==============================] - 254s 10s/step - loss: 0.0359 - accuracy: 0.9871 - val_loss: 0.0273 - val_accuracy: 0.9971
Epoch 3/10
25/25 [==============================] - 255s 10s/step - loss: 0.0309 - accuracy: 0.9897 - val_loss: 0.0269 - val_accuracy: 0.9971
Epoch 4/10
25/25 [==============================] - 254s 10s/step - loss: 0.0403 - accuracy: 0.9858 - val_loss: 0.0233 - val_accuracy: 0.9971
Epoch 5/10
25/25 [==============================] - 256s 10s/step - loss: 0.0404 - accuracy: 0.9819 - val_loss: 0.0215 - val_accuracy: 0.9985
Epoch 6/10
25/25 [==============================] - 202s 8s/step - loss: 0.0264 - accuracy: 0.9948 - val_loss: 0.0228 - val_accuracy: 0.9985
Epoch 7/10
25/25 [==============================] - 253s 10s/step - loss: 0.0342 - accuracy: 0.9884 - val_loss: 0.0202 - val_accuracy: 0.9985
Epoch 8/

In [61]:
loss, accuracy = model.evaluate(validation_generator)
print('Validation accuracy:', accuracy)

22/22 [==============================] - 93s 4s/step - loss: 0.0174 - accuracy: 0.9985
Validation accuracy: 0.9985486268997192


In [62]:
model_path = '/content/drive/MyDrive/inceptionv3.h5'
model.save(model_path)